In [1]:
# import libraries
import os
import pandas as pd
import datetime as dt

In [2]:
# must match a reference ontology
# path in final version will be /opt/data/dim/
concept = pd.read_csv('volumes/data/dim/CONCEPT.csv.gz', header=0, sep='\t', low_memory=False)

In [3]:
concept = concept[['concept_code', 'concept_id']]

In [4]:
concept.drop_duplicates(inplace=True)

In [5]:
concept.rename(columns= {'concept_code': 'cpt'}, inplace=True)

In [15]:
# make a control file to iterate through
# path in final version will be /opt/data/raw/
path = 'volumes/data/raw/'
files = os.listdir(path)
csv_files = [x for x in files if x.endswith(".xlsx")]

# removes '.xlsx' (5 characters)
hospital_ids = [int(x[:-5]) for x in csv_files] 
control = pd.DataFrame({'file': csv_files, 'hospital_id': hospital_ids})

In [16]:
#cross reference the control file with the static dimension table
# path in final version will be /opt/dta/dim/
dim = pd.read_csv('volumes/data/dim/hospital.csv', usecols=['hospital_id', 'affiliation'])
control = control.merge(dim, how= 'left', on='hospital_id')
control.sort_values(by='hospital_id', inplace=True, ignore_index=True)

In [18]:
## loop (PENDING)

In [19]:
# print file you are working on now
print(dt.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ' - parsing hospital - ' + str(control.hospital_id[0]))

2021-09-20 16:34:22 - parsing hospital - 9


In [58]:
# read in the data
# replace 0 with i in loop
df = pd.read_excel('volumes/data/raw/' + control.file[0])

In [77]:
out = pd.DataFrame({
    'cpt': df['HCPCS'],
    'gross': df['CHARGE']
})
out.drop_duplicates(inplace=True)

In [80]:
# if out is not Null loop

In [81]:
# uniform
out.dropna(subset=['cpt'], inplace=True)
out = out[out.cpt != '']
out = out[out.cpt != '*']

In [90]:
# merge was skipping CPT values with no letters on them
out['cpt'] = out['cpt'].astype(str)

In [95]:
# must match a reference code, usually CPT or HCPCS
out = out.merge(concept, on='cpt', sort=True)

In [100]:
out = out.drop('cpt', axis=1)

In [102]:
# melt from wide to long, and clean
long = pd.melt(out, id_vars='concept_id')

In [105]:
if long.value.dtypes not in ('float64', 'int64'):
    long['value'] = long['value'].str.strip()
    long['value'] = long['value'].str.replace(',', '', regex=False)
    long['value'] = long['value'].str.replace('[$]', '', regex=False)
    long['value'] = long.value.astype(float)
long.dropna(subset=['value'], inplace=True)
long = long[long.value > 0]

# Add hospital id and order columns
# later change the 0 for i inside the loop
long['hospital_id'] = control.hospital_id[0]
long = long[long.columns[[3,0,1,2]]]

In [107]:
# if out is not Null clause

In [108]:
# write the data to a flatfile for postgres
# path will be later /opt/data/transformed/
# replace 0 with i inside the loop
if long.shape[0] > 0:
    out_path = 'volumes/data/transformed/' + str(control.hospital_id[0]) + '.csv'
    long.to_csv(out_path, header=False, index=None)

In [109]:
# clear variables from last iteration (inside the loop)
# del out
# del long

In [110]:
col_names = ['hospital_id', 'concept_id', 'variable', 'value']
dfr = pd.read_csv('volumes/data/transformed/9.csv', header=None, names=col_names)

In [111]:
dfr.head()

,hospital_id,concept_id,variable,value
0,9,709849,gross,341.25
1,9,2101701,gross,100.00
2,9,927066,gross,805.00
3,9,42742484,gross,2745.00
4,9,42742485,gross,1375.00


In [112]:
dfn = pd.read_csv('../../NC/hospital-price-transparency/volumes/data/transformed/9.csv', header=None, names=col_names)

In [113]:
dfn.head()

,hospital_id,concept_id,variable,value
0,9,709849,gross,341.25
1,9,2101701,gross,100.00
2,9,927066,gross,805.00
3,9,42742484,gross,2745.00
4,9,42742485,gross,1375.00


In [117]:
dfn.loc[lambda df: df['concept_id'] == 42742484]

,hospital_id,concept_id,variable,value
3,9,42742484,gross,2745.0


In [116]:
dfr.loc[lambda df: df['concept_id'] == 42742484]

,hospital_id,concept_id,variable,value
3,9,42742484,gross,2745.0


In [118]:
print(dfn.shape, dfr.shape)

(2540, 4) (2540, 4)
